#### Removing spatially autocorrelated training and validation points

This scripts removes points that are less than 30m from each other and share status as val/training and are same location-date

Initially used the Near tool with val/training and location-date as match fields to prep the point data for running here, but that only matches points with the closest point (not all points under 30m away)

Pivoted to spatial join output, with val/training split and location-date as match fields

In [10]:

import pandas as pd

# Load the file of points that are less than 30m apart from another point on the same date/location
# does also include clusters of more than two points
# didn't work well to join back to full dataset, so I recommend only using the full point file below
df = pd.read_csv("data/AllTraining_080425prj_closepts.csv")

# load the file with all training points with near distance and near feature id
all = pd.read_csv("data/AllTraining_080425prj.csv")

# file with spatial join output, including points matched to themselves
sj = pd.read_csv("data/AllTraining_080425prj_SpatialJoin.csv")

# Set the threshold distance (m)
threshold = 30

# all

#### Pull in spatial join dataset, remove points matched to themselves

In [ ]:
# Remove points in sj where target ID matches join ID - matched to itself
sj_filter = sj[sj['TARGET_FID'] != sj['JOIN_FID']]

print(f"Removed {len(sj) - len(sj_filter)} self-matched points, keeping {len(sj_filter)} point pairs.")

# Track IDs to delete and processed pairs
delete_ids = set()
processed_pairs = set()

# Count how many times each ID appears as TARGET_FID and add that back as a field to each row with that Target_FID
target_counts = sj_filter['TARGET_FID'].value_counts().to_dict()
sj_filter.loc[:, 'Target_Count'] = sj_filter['TARGET_FID'].map(target_counts)
# count how many times each ID appears as JOIN_FID and add that back as a field to each row with that Join_FID
join_counts = sj_filter['JOIN_FID'].value_counts().to_dict()
sj_filter.loc[:, 'Join_Count'] = sj_filter['JOIN_FID'].map(join_counts)

# Display the first couple rows with all columns showing
pd.set_option('display.max_columns', None)
display(sj_filter.head(8))

# FYI that join_ID is the one that matches up with orig_FID
# all fields in this table (including duplicate fields) match the joined feature, not the target/original feature

# Iterate through pairs
for _, row in sj_filter.iterrows():
    t_id = int(row['TARGET_FID'])
    j_id = int(row['JOIN_FID'])
    pair = tuple(sorted([t_id, j_id]))
    target_count = int(row['Target_Count'])
    join_count = int(row['Join_Count'])

    # Skip this pair if either ID is already marked for deletion or pair is processed
    if t_id in delete_ids or j_id in delete_ids or pair in processed_pairs:
        continue

    # Preferentially delete the ID matched with more other points
    if target_count > join_count:
        delete_ids.add(t_id)
    elif join_count > target_count:
        delete_ids.add(j_id)
    else:
        # If counts are equal, delete the higher ID
        delete_ids.add(max(pair))
    processed_pairs.add(pair)

print(f"Marked {len(delete_ids)} points for deletion based on spatial matches.")

# Optionally, filter out deleted points from the original dataset
filtered_sj_df = all[~all['Orig_FID'].isin(delete_ids)]
filtered_sj_df.head()

print(f"Filtered dataset now has {len(filtered_sj_df)} points after removing {len(delete_ids)} spatially autocorrelated points.")

# save filtered dataset as csv
# filtered_sj_df.to_csv("data/AllTraining_080425prj_SJfiltered.csv")


Removed 4997 self-matched points, keeping 3554 point pairs.


C:\Users\annie.taylor\AppData\Local\Temp\ipykernel_18488\2840446142.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sj_filter.loc[:, 'Target_Count'] = sj_filter['TARGET_FID'].map(target_counts)


,Join_Count,TARGET_FID,JOIN_FID,Location,Date,Type,Region,Latitude,Longitude,Split,Date_Loc,NEAR_FID,NEAR_DIST,Orig_FID,Location_1,Date_1,Type_1,Region_1,Latitude_1,Longitude_1,Split_1,Date_Loc_1,NEAR_FID_1,NEAR_DIST_1,Orig_FID_1,Target_Count
226,1,226,254,Oxnard,11/9/2023,hoop,NaN,34.241627,-119.132327,1,2023-11-09 Oxnard,226.0,26.160596,254,Oxnard,11/9/2023,hoop,NaN,34.241627,-119.132327,1,2023-11-09 Oxnard,226.0,26.160596,254,1
236,1,235,320,Oxnard,5/6/2023,hoop,NaN,34.151062,-119.104499,1,2023-05-06 Oxnard,235.0,28.005998,320,Oxnard,5/6/2023,hoop,NaN,34.151062,-119.104499,1,2023-05-06 Oxnard,235.0,28.005998,320,1
245,1,243,245,Oxnard,10/14/2019,hoop,NaN,34.189403,-119.123544,2,2019-10-14 Oxnard,243.0,29.336011,245,Oxnard,10/14/2019,hoop,NaN,34.189403,-119.123544,2,2019-10-14 Oxnard,243.0,29.336011,245,2
246,1,243,305,Oxnard,10/14/2019,hoop,NaN,34.189648,-119.123690,2,2019-10-14 Oxnard,243.0,28.300004,305,Oxnard,10/14/2019,hoop,NaN,34.189648,-119.123690,2,2019-10-14 Oxnard,243.0,28.300004,305,2
248,2,245,243,Oxnard,10/14/2019,hoop,NaN,34.189632,-119.123383,2,2019-10-14 Oxnard,305.0,28.300004,243,Oxnard,10/14/2019,hoop,NaN,34.189632,-119.123383,2,2019-10-14 Oxnard,305.0,28.300004,243,1
254,2,249,281,Oxnard,11/9/2023,hoop,NaN,34.232007,-119.136027,1,2023-11-09 Oxnard,264.0,26.837272,281,Oxnard,11/9/2023,hoop,NaN,34.232007,-119.136027,1,2023-11-09 Oxnard,264.0,26.837272,281,1
257,1,251,282,Oxnard,11/9/2023,hoop,NaN,34.231686,-119.135760,1,2023-11-09 Oxnard,251.0,27.541831,282,Oxnard,11/9/2023,hoop,NaN,34.231686,-119.135760,1,2023-11-09 Oxnard,251.0,27.541831,282,1
260,1,254,226,Oxnard,11/9/2023,hoop,NaN,34.241722,-119.132067,1,2023-11-09 Oxnard,254.0,26.160596,226,Oxnard,11/9/2023,hoop,NaN,34.241722,-119.132067,1,2023-11-09 Oxnard,254.0,26.160596,226,1


Marked 875 points for deletion based on spatial matches.
Filtered dataset now has 4122 points after removing 875 spatially autocorrelated points.


#### Pull in full dataset, remove points that are too close together
Preferentially remove points that are paired with more than one other point 

In [ ]:
# Track pairs and points to delete
delete_ids = set()
processed_pairs = set()

# Iterate through rows
for _, row in all.iterrows():
    oid = row['Orig_FID']
    near_fid = row['NEAR_FID']
    near_dist = row['NEAR_DIST']

    if pd.notnull(near_fid) and near_dist < threshold:
        pair = tuple(sorted([int(oid), int(near_fid)]))
        # If either oid or near_fid is already marked for deletion, skip this pair
        #  as it has already been remedied 
        if int(oid) in delete_ids or int(near_fid) in delete_ids:
            pass
        else:
            # Check if oid or near_fid appears elsewhere in NEAR_FID column (excluding this row)
            oid_in_near_fid = ((all['NEAR_FID'] == int(oid)) & (all.index != row.name)).any()
            near_fid_in_near_fid = ((all['NEAR_FID'] == int(near_fid)) & (all.index != row.name)).any()
            # if oid appears in NEAR_FID but near_fid does not, delete oid
            if oid_in_near_fid and not near_fid_in_near_fid:
                delete_ids.add(int(oid))
            # if near_fid appears in NEAR_FID but oid does not, delete near_fid
            elif near_fid_in_near_fid and not oid_in_near_fid:
                delete_ids.add(int(near_fid))
            else:
                # If both or neither appears elsewhere, delete the point with higher OID
                delete_ids.add(max(pair))
        processed_pairs.add(pair) 


# Filter out the points to delete
filtered_df = all[~all['Orig_FID'].isin(delete_ids)]

# Save the cleaned data
filtered_df.to_csv("data/AllTraining_080425prj_FilteredPts.csv")

print(f"Deleted {len(delete_ids)} points within {threshold} meters of another point.")
filtered_df


# save the points that were deleted as a csv
# deleted_df = df[df['Orig_FID'].isin(delete_ids)]
# deleted_df.to_csv("data/AllTraining_080425prj_DeletedPts.csv")

Deleted 744 points within 30 meters of another point.


,Location,Date,Type,Region,Latitude,Longitude,Split,Date_Loc,NEAR_FID,NEAR_DIST,Orig_FID
0,CA,5/26/2023,other,Morgan Hill,37.188061,-121.725901,1,2023-05-26 CA,NaN,NaN,1
1,CA,5/26/2023,other,Morgan Hill,37.189068,-121.724510,1,2023-05-26 CA,NaN,NaN,2
2,CA,5/26/2023,other,Morgan Hill,37.190223,-121.722561,1,2023-05-26 CA,NaN,NaN,3
3,CA,5/26/2023,other,Morgan Hill,37.187451,-121.719603,1,2023-05-26 CA,NaN,NaN,4
4,CA,5/26/2023,other,Morgan Hill,37.189402,-121.726455,1,2023-05-26 CA,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...
4992,Santa Maria,2/6/2021,mulch,NaN,34.990410,-120.497203,2,2021-02-06 Santa Maria,NaN,NaN,5899
4993,Santa Maria,2/6/2021,mulch,NaN,34.990676,-120.501473,2,2021-02-06 Santa Maria,NaN,NaN,5900
4994,Santa Maria,2/6/2021,mulch,NaN,34.989570,-120.501519,2,2021-02-06 Santa Maria,NaN,NaN,5901
4995,Santa Maria,2/6/2021,mulch,NaN,34.990623,-120.502547,2,2021-02-06 Santa Maria,NaN,NaN,5902


In [ ]:
# Original method that doesn't preferentially delete points with more than one pair
# or account for points already marked for deletion

# Track pairs and points to delete
delete_ids = set()
processed_pairs = set()

# Iterate through rows
for _, row in df.iterrows():
    oid = row['Orig_FID']
    near_fid = row['NEAR_FID']
    near_dist = row['NEAR_DIST']

    if pd.notnull(near_fid) and near_dist < threshold:
        pair = tuple(sorted([int(oid), int(near_fid)]))
        if pair not in processed_pairs:
            delete_ids.add(max(pair))  # delete the one with higher OID
            processed_pairs.add(pair)

# Filter out the points to delete
filtered_df = df[~df['Orig_FID'].isin(delete_ids)]

print(f"Deleted {len(delete_ids)} points within {threshold} meters of another point.")

# export csv of deleted points
deleted_df = df[df['Orig_FID'].isin(delete_ids)]
deleted_df.to_csv("data/AllTraining_080425prj_DeletedPts2.csv")


Deleted 941 points within 30 meters of another point.
